In [ ]:
import numpy as np
from matplotlib import pyplot as plt
# from sfdata import SFScanInfo
from sfdata import SFDataFiles, SFScanInfo, SFProcFile

import pickle
import matplotlib.patches as patches

%matplotlib widget

import numpy as np
import json, h5py
import os, glob
import time
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm
from IPython.display import clear_output, display
import matplotlib.colors as colors
import matplotlib.ticker as ticker
# from ipyfilechooser import FileChooser

import warnings
warnings.filterwarnings('ignore')

from sfdata import SFDataFile, SFDataFiles
from sys import getsizeof
import ipywidgets as widgets
from ipywidgets import *
from scipy import signal, optimize

from scipy.signal import savgol_filter

In [ ]:
def mm_to_fs(mm):
    return 3335.6*2*mm


In [ ]:
#put run numbers. if more than one run, write [21, 22, 24, 30]. THey will be merged if the scan was the same PV
run_numbers = [15]#np.arange(1829,2017+1)#[740]
run_numbers

In [ ]:
#################### Load data #########################
### Can start loading while run is still ongoing   #######

from os import listdir
from os.path import isfile, join

path = "/sf/maloja/data/p21625/raw/"

file_list = os.listdir(path)


scan_name_list = []

for i in run_numbers:
    s = '%04d'%i

    for f in file_list:
        if s in f:
            #fn = "/sf/maloja/data/p21235/raw/%s/meta/scan.json"%s
            scan_name_list.append(f)
                  
for ii , scan_name in enumerate(scan_name_list):
    scan_name_list[ii] = path+scan_name
scan_name_list

In [ ]:
### Get amps value from files



In [ ]:
# channel = ['SATUN14-MBND100:I-READ']
# current_list = []

# for scan_name in scan_name_list:
#     # print(scan_name)
#     scan = SFScanInfo(scan_name + "/meta/scan.json")
#     # print(scan.readbacks)
#     for i, step in enumerate(scan[:]):
#         current = None
        
#         subset = step[channel]
#         # subset.drop_missing()
#         # print(step.keys())
#         scan_steps = scan.readbacks
#         current_list.append(step[channel[0]].data.mean())


In [ ]:

channels = ["SATES21-GES1:A2_VALUES", "SATES21-GES1:A4_VALUES"]



tof = []
apd = []

scan_var = []
delay_A = []

for counter, scan_name in enumerate(scan_name_list):
    print(scan_name)
    scan = SFScanInfo(scan_name + "/meta/scan.json")
    print(scan.readbacks)
    
    # print(scan[0].keys())
    vmi_step = []
    vmi_radial_temp = []
    I0_step = []

    for i, step in enumerate(scan[:]):
        temp_tof = None
        temp_apd = None
        try:
            subset = step[channels]
            subset.drop_missing()
            # print(step.keys())
            scan_steps = scan.readbacks
            # print(i)
            temp_apd = subset["SATES21-GES1:A4_VALUES"].data[:,0:5000]
            #print(temp_apd.shape)
            temp_tof = subset["SATES21-GES1:A2_VALUES"].data[:,0:10000]
        except:
            print('error')
            continue
            
        if temp_tof is not None and temp_apd is not None :
            tof.append(temp_tof)
            apd.append(temp_apd)
            delay_A.append(scan_steps[i]*np.ones(temp_tof.shape[0]))

tof = -1*np.vstack(tof)
print(tof.shape)
apd = np.vstack(apd)
print(apd.shape)
step  = scan.readbacks
scan_var = np.hstack(delay_A)
print(scan_var.shape)

In [ ]:
tof.shape

In [ ]:
apd.shape

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.plot(apd.mean(0))
plt.subplot(1,2,2)
plt.plot(tof.mean(0))

plt.show()

In [ ]:
# clean spectra:

clean_apd = np.copy(apd)

clean_apd = apd - (np.mean(apd[:,0:1000],1)*np.ones_like(apd).T).T

clean_tof = np.copy(tof)

clean_tof = tof - (np.mean(tof[:,0:1000],1)*np.ones_like(tof).T).T

In [ ]:
plt.figure()
plt.plot(clean_tof[-1000])
# plt.plot(tof[-100])
plt.show()

In [ ]:
apd_int = np.sum(np.abs(clean_apd[:,1096:1250]),1)
tof_int = np.sum(np.abs(clean_tof[:,4970:5200]),1)
# tof_int = np.sum(np.abs(tof[:,6380:6450]),1)

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.hist(tof_int[::2], bins = 100)
plt.subplot(1,2,2)

plt.hist(tof_int[1::2], bins = 100)

plt.show()

In [ ]:
np.arange(0,10)[::2],np.arange(0,10)[1::2]

In [ ]:
plt.figure()
plt.suptitle(scan_name_list[0][24:])

plt.subplot(2,2,1)
plt.hist2d(apd_int, tof_int, bins = 300)
plt.subplot(2,2,2)
plt.title('APD')
plt.hist(apd_int, bins = 300)
plt.subplot(2,2,3)
plt.title('TOF')

plt.hist(tof_int, bins = 300)
plt.tight_layout()
plt.show()

plt.figure()
plt.suptitle(scan_name_list[0][24:])

plt.hist2d(apd_int[apd_int >8e4], tof_int[apd_int >8e4], bins = 300)
plt.xlabel('APD')
plt.xlabel('TOF')

# plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
plt.suptitle(scan_name_list[0])
plt.subplot(2,1,1)
plt.plot( apd_int, '.-',label = 'apd')
plt.legend()


plt.plot(savgol_filter(apd_int,31,2))
plt.subplot(2,1,2)
plt.plot(tof_int,'.-', label = 'TOF')
# plt.xlabel(scan.parameters['Id'] + scan.parameters['units'])
plt.legend()
plt.show()

In [ ]:
spec = []
std = []
I0 = []
for s in np.arange(0,len(apd_int),1500):
    absorption = apd_int[s:s+1500]
    I0.append(tof_int[s:s+1500].mean())
    spec.append(absorption.mean())
    std.append(absorption.std())
    
plt.figure()
plt.suptitle(scan_name_list[0])
plt.subplot(2,1,1)
plt.plot( np.asarray(spec), '.-',label = 'absorption')

plt.subplot(2,1,2)
plt.plot( I0,'.-', label = 'I0')
# plt.xlabel(scan.parameters['Id'] + scan.parameters['units'])
plt.legend()
plt.show()



In [ ]:
# step = np.unique(scan_var)

In [ ]:
spec = []
std = []
I0 = []
for s in step:
    mask = (scan_var == s) & (tof_int > 3.3e4) #& (tof_int < 6e5)
    absorption = apd_int[mask] / tof_int[mask]
    I0.append(tof_int[mask].mean())
    spec.append(absorption.mean())
    std.append(absorption.std())
    


In [ ]:
plt.figure()
plt.suptitle(scan_name_list[0])
# plt.subplot(2,1,1)
plt.plot(step, spec, '.-',label = '')
# plt.errorbar(step, spec, np.asarray(std), label = 'std')

# plt.subplot(2,1,2)
# plt.plot(step, I0,'.-', label = 'I0')
plt.xlabel(scan.parameters['Id'] + scan.parameters['units'])
plt.legend()
plt.show()

In [ ]:
0.06/2.1

In [ ]:
# plt.figure()
# plt.title(scan_name_list[0])


# for pe in np.arange(2e5,8e5,1e5):
    
#     spec = []
#     std = []
#     for s in step:
#         mask = (scan_var == s) & (tof_int > pe)  & (tof_int < pe+1e5)
#         absorption = apd_int[mask] / tof_int[mask]
#         spec.append(absorption.mean())
#         std.append(absorption.std())
    
#     # plt.plot(step, spec)
#     plt.errorbar(step, spec, np.asarray(std), label = 'TOF inten. %d'%(pe+1e5/2))
    
# plt.xlabel(scan.parameters['Id'] + scan.parameters['units'])

# plt.legend()
# plt.show()

In [ ]:
# spec = []
# for s in step:
#     mask = (scan_var == s)


#     plt.figure()
#     plt.subplot(2,2,1)
#     plt.hist2d(apd_int[mask], tof_int[mask], bins = 300)
#     plt.subplot(2,2,2)
#     plt.title('APD')
#     plt.hist(apd_int[mask], bins = 300)
#     plt.subplot(2,2,3)
#     plt.title('TOF')

#     plt.hist(tof_int[mask], bins = 300)
#     plt.show()

In [ ]:
# plt.figure(figsize = (10,10))
# pltn = np.round(np.sqrt(step.shape[0]))+1
# for i,s in enumerate(step):
#     mask = (scan_var == s)
#     plt.subplot(pltn,pltn,i+1)
#     plt.suptitle('Intensity corr. signal/100000  pulse ene.3.5mJ mono 0.5eV @ 710eV')
#     plt.title('Ene. %0.2f corr %0.4f' %(step[i], np.corrcoef(apd_int[mask], tof_int[mask])[0,1]))
#     plt.hist2d(apd_int[mask]/100000, tof_int[mask]/100000, bins = 100)
# plt.tight_layout()
# plt.show()

In [ ]:
150*0.1*(0.5/7)